# Draw flows between nodes as a network

### Load data

In [2]:
# Load data
import pandas as pd
moves = pd.read_csv('data-raw/ED_bed_moves_raw_2022-03-21.csv')

In [3]:
moves.head()

,csn,location_string,admission,discharge,location
0,1017787509,ED^null^null,2020-02-04T13:58:58Z,2020-02-04T14:38:33Z,ED Waiting
1,1017787509,ED^UCHED ADULT TRIAGE^NONE,2020-02-04T14:38:33Z,2020-02-04T16:49:23Z,ED TRIAGE
2,1017787509,ED^UCHED UTC11^11-UTC,2020-02-04T16:49:23Z,2020-02-04T17:36:21Z,ED UTC
3,1017787509,ED^UCHED UTC06^06-UTC,2020-02-04T17:36:21Z,2020-02-04T17:53:28Z,ED UTC
4,1017787509,ED^UCHED ADULT TRIAGE^NONE,2020-02-04T17:53:28Z,2020-02-04T18:43:00Z,ED TRIAGE


### Calculate next location

In [4]:
moves['next_location'] = moves.groupby('csn')['location'].shift(-1)
moves['next_csn'] = moves['csn'].astype(str).shift(-1)
moves.head(10)

,csn,location_string,admission,discharge,location,next_location,next_csn
0,1017787509,ED^null^null,2020-02-04T13:58:58Z,2020-02-04T14:38:33Z,ED Waiting,ED TRIAGE,1017787509
1,1017787509,ED^UCHED ADULT TRIAGE^NONE,2020-02-04T14:38:33Z,2020-02-04T16:49:23Z,ED TRIAGE,ED UTC,1017787509
2,1017787509,ED^UCHED UTC11^11-UTC,2020-02-04T16:49:23Z,2020-02-04T17:36:21Z,ED UTC,ED UTC,1017787509
3,1017787509,ED^UCHED UTC06^06-UTC,2020-02-04T17:36:21Z,2020-02-04T17:53:28Z,ED UTC,ED TRIAGE,1017787509
4,1017787509,ED^UCHED ADULT TRIAGE^NONE,2020-02-04T17:53:28Z,2020-02-04T18:43:00Z,ED TRIAGE,NaN,1027701584
5,1027701584,ED^null^null,2021-05-22T07:37:55Z,2021-05-22T07:43:00Z,ED Waiting,ED RAT,1027701584
6,1027701584,ED^UCHED RAT CHAIR^RAT-CHAIR,2021-05-22T07:43:00Z,2021-05-22T07:52:00Z,ED RAT,ED RAT,1027701584
7,1027701584,ED^UCHED RAT06^RAT-06,2021-05-22T07:52:00Z,2021-05-22T08:15:00Z,ED RAT,ED MAJORS,1027701584
8,1027701584,ED^NON COVID MAJORS 22^22-NON COVID MAJORS,2021-05-22T08:15:00Z,2021-05-22T09:40:00Z,ED MAJORS,NaN,1029080319
9,1029080319,ED^null^null,2021-07-27T07:39:13Z,2021-07-27T07:39:14Z,ED Waiting,ED Waiting,1029080319


In [5]:
# Where next_location is NaN, set it to 'Left'
moves['next_location'] = moves['next_location'].fillna('Left')

### Create an edge list

In [6]:
edge_list = moves.groupby(['location', 'next_location']).size().reset_index(name='frequency')
edge_list.head(20)

,location,next_location,frequency
0,ED CDU,ED CDU,180
1,ED CDU,ED MAJORS,94
2,ED CDU,ED RAT,2
3,ED CDU,ED RESUS,2
4,ED CDU,ED TAF,1
5,ED CDU,ED TRIAGE,1
6,ED CDU,ED UTC,9
7,ED CDU,ED Waiting,1
8,ED CDU,Left,2195
9,ED CDU,OTF,17


In [7]:
### Save the edge list

In [8]:
edge_list.to_csv('edge_list.csv', index=False)

### Plot the network

First step is to install the plotting module (Note - pygraphviz is tricky to install and doesn't work in this Jupyter environment)

In [24]:
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 2.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/pygraphviz
      copying pygraphviz/__init__.py -> build/lib.linux-x86_64-cpython-310/pygraphviz
      copying pygraphviz/agraph.py -> build/lib.linux-x86_64-cpython-310/pygraphviz
      copying pygraphviz/graphviz.py -> build/lib.linux-x86_64-cpython-310/pygraphviz
      copying pygraphviz/scraper.py -> build/lib.linux-x86_64-cpython-310/pygraphviz
      copying pygraphviz/testing.py -> build/lib.linux-x86_64-cpython-310/p